In [2]:
#### import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import SGDClassifier, Perceptron #doubt linear models work, linear reg.
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
from sklearn.model_selection import train_test_split
import numpy.random as nr
nr.seed = 42

In [4]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os import path as op
import itertools
import re
import xarray as xr

%matplotlib inline

In [5]:
inputPath = '/scratch/adomakor412/all_npy3'
PATH = os.path.expanduser(inputPath)
ncPath = os.path.expanduser('/scratch/adomakor412/april_data_2019')

In [6]:
columns = [
    "time",
    "longitude",
    "latitude",
    #"band",#Necessary?
    "G17_Temp",
    "G17_mean",
    "G17_std",#mean and std outside inner for loop for comp. eff.
    "target_G16_Temp"
]

df = pd.DataFrame({},columns = columns)
df

,time,longitude,latitude,G17_Temp,G17_mean,G17_std,target_G16_Temp


In [7]:
def longitude(lonMin, lonMax, col, colSize):
    lon = (col/colSize)* (lonMax - lonMin)
    return lon

def latitude(latMin, latMax, row, rowSize):
    lat = (row/rowSize)* (latMax - latMin)
    return lat

def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    invRad = np.array(rad)**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2) 
    return T

In [ ]:
extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]

    
for (bb,dd) in list(itertools.product([8],range(11))):
    DD = str(98+dd).zfill(3)
    lookup = f'M6C08_G17_s2019{DD}'
    ncFiles = [f for f in os.listdir(ncPath) if re.search(lookup,f)]
    npFiles = [f for f in os.listdir( PATH ) if re.search(lookup,f)]

    for ncf, npf in zip(ncFiles,npFiles):
        imageBox = np.load(op.join( PATH,npf))
        myFile = xr.open_dataset(op.join(ncPath,ncf))
        planck_fk1 = float(myFile['planck_fk1'].data)
        planck_fk2 = float(myFile['planck_fk2'].data) 
        planck_bc1 = float(myFile['planck_bc1'].data)                       
        planck_bc2 = float(myFile['planck_bc2'].data)

        time = ncf[31:38]
        G17_mean = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
        G17_std = Rad2BT(imageBox.std(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
        
        hh = ncf[34:36]
        mm = ncf[36:38]
        
        G16_npy = np.load( op.join(PATH, npf.replace('G16','G17',1)) )
        G16_ncf = xr.open_dataset(op.join( ncPath, ncf.replace('G16','G17',1) ))
        G16_fk1 = float(G16_ncf['planck_fk1'].data)
        G16_fk2 = float(G16_ncf['planck_fk2'].data) 
        G16_bc1 = float(G16_ncf['planck_bc1'].data)                       
        G16_bc2 = float(G16_ncf['planck_bc2'].data)
        
        target_G16_Temp = Rad2BT(G16_npy, G16_fk1, G16_fk2, G16_bc1, G16_bc2)
        x,y = imageBox.shape[0],imageBox.shape[1]
        for i,j in itertools.product(range(x),range(y)):
            lon = longitude( extent_pc[0], extent_pc[1], i, x )
            lat = latitude( extent_pc[2], extent_pc[3], j, y)
            
            G17_Temp = Rad2BT(imageBox[i,j], planck_fk1, planck_fk2, planck_bc1, planck_bc2)#unfiltered
            target_G16_Temp = Rad2BT( G16_npy[i,j], G16_fk1, G16_fk2, G16_bc1, G16_bc2 )
            
            df.append([time, lon, lat, G17_Temp, G17_mean, G17_std, target_G16_Temp])
            print([time, lon, lat, G17_Temp, G17_mean, G17_std, target_G16_Temp])
    
# df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", names=columns)
# for column in df.columns:
#     if df[column].dtype == "object":
#         df[column] = df[column].str.strip()
# df = df.dropna()

['0980700', 0.0, -0.0, 248.38452155230846, 244.17938379094736, 201.81719599208182, 248.38452155230846]
['0980700', 0.0, -0.044621321695760593, 248.38452155230846, 244.17938379094736, 201.81719599208182, 248.38452155230846]
['0980700', 0.0, -0.08924264339152119, 248.0350599376521, 244.17938379094736, 201.81719599208182, 248.0350599376521]
['0980700', 0.0, -0.1338639650872818, 248.0350599376521, 244.17938379094736, 201.81719599208182, 248.0350599376521]
['0980700', 0.0, -0.17848528678304237, 247.99112126305047, 244.17938379094736, 201.81719599208182, 247.99112126305047]
['0980700', 0.0, -0.22310660847880298, 248.25389641960302, 244.17938379094736, 201.81719599208182, 248.25389641960302]
['0980700', 0.0, -0.2677279301745636, 248.51464805471505, 244.17938379094736, 201.81719599208182, 248.51464805471505]
['0980700', 0.0, -0.31234925187032414, 248.21024174991246, 244.17938379094736, 201.81719599208182, 248.21024174991246]
['0980700', 0.0, -0.35697057356608475, 248.21024174991246, 244.179383

['0980700', 0.0, -3.4804630922693263, 247.90307212243414, 244.17938379094736, 201.81719599208182, 247.90307212243414]
['0980700', 0.0, -3.5250844139650868, 248.21024174991246, 244.17938379094736, 201.81719599208182, 248.21024174991246]
['0980700', 0.0, -3.5697057356608477, 248.21024174991246, 244.17938379094736, 201.81719599208182, 248.21024174991246]
['0980700', 0.0, -3.614327057356608, 248.0350599376521, 244.17938379094736, 201.81719599208182, 248.0350599376521]
['0980700', 0.0, -3.6589483790523687, 247.99112126305047, 244.17938379094736, 201.81719599208182, 247.99112126305047]
['0980700', 0.0, -3.703569700748129, 247.99112126305047, 244.17938379094736, 201.81719599208182, 247.99112126305047]
['0980700', 0.0, -3.74819102244389, 247.94712570898017, 244.17938379094736, 201.81719599208182, 247.94712570898017]
['0980700', 0.0, -3.7928123441396506, 247.94712570898017, 244.17938379094736, 201.81719599208182, 247.94712570898017]
['0980700', 0.0, -3.837433665835411, 247.99112126305047, 244.1

['0980700', 0.0, -8.388808478802991, 246.5539785994801, 244.17938379094736, 201.81719599208182, 246.5539785994801]
['0980700', 0.0, -8.433429800498752, 246.59983403957187, 244.17938379094736, 201.81719599208182, 246.59983403957187]
['0980700', 0.0, -8.478051122194513, 246.59983403957187, 244.17938379094736, 201.81719599208182, 246.59983403957187]
['0980700', 0.0, -8.522672443890274, 246.59983403957187, 244.17938379094736, 201.81719599208182, 246.59983403957187]
['0980700', 0.0, -8.567293765586035, 246.59983403957187, 244.17938379094736, 201.81719599208182, 246.59983403957187]
['0980700', 0.0, -8.611915087281794, 246.5539785994801, 244.17938379094736, 201.81719599208182, 246.5539785994801]
['0980700', 0.0, -8.656536408977555, 246.5539785994801, 244.17938379094736, 201.81719599208182, 246.5539785994801]
['0980700', 0.0, -8.701157730673316, 246.59983403957187, 244.17938379094736, 201.81719599208182, 246.59983403957187]
['0980700', 0.0, -8.745779052369077, 246.59983403957187, 244.179383790

['0980700', 0.0, -12.449348753117205, 246.3699249303876, 244.17938379094736, 201.81719599208182, 246.3699249303876]
['0980700', 0.0, -12.493970074812967, 246.3699249303876, 244.17938379094736, 201.81719599208182, 246.3699249303876]
['0980700', 0.0, -12.538591396508727, 247.23529799515927, 244.17938379094736, 201.81719599208182, 247.23529799515927]
['0980700', 0.0, -12.58321271820449, 246.73702574257743, 244.17938379094736, 201.81719599208182, 246.73702574257743]
['0980700', 0.0, -12.627834039900248, 246.78262879133797, 244.17938379094736, 201.81719599208182, 246.78262879133797]
['0980700', 0.0, -12.672455361596008, 246.78262879133797, 244.17938379094736, 201.81719599208182, 246.78262879133797]
['0980700', 0.0, -12.71707668329177, 246.78262879133797, 244.17938379094736, 201.81719599208182, 246.78262879133797]
['0980700', 0.0, -12.76169800498753, 246.78262879133797, 244.17938379094736, 201.81719599208182, 246.78262879133797]
['0980700', 0.0, -12.806319326683292, 246.82817240947176, 244.1

['0980700', 0.0, -17.40231546134663, 246.73702574257743, 244.17938379094736, 201.81719599208182, 246.73702574257743]
['0980700', 0.0, -17.446936783042393, 246.73702574257743, 244.17938379094736, 201.81719599208182, 246.73702574257743]
['0980700', 0.0, -17.491558104738154, 247.1903052886279, 244.17938379094736, 201.81719599208182, 247.1903052886279]
['0980700', 0.0, -17.536179426433915, 247.1903052886279, 244.17938379094736, 201.81719599208182, 247.1903052886279]
['0980700', 0.0, -17.580800748129676, 247.14525094318788, 244.17938379094736, 201.81719599208182, 247.14525094318788]
['0980700', 0.0, -17.625422069825436, 247.1903052886279, 244.17938379094736, 201.81719599208182, 247.1903052886279]
['0980700', 0.0, -17.670043391521194, 247.23529799515927, 244.17938379094736, 201.81719599208182, 247.23529799515927]
['0980700', 0.0, -17.714664713216955, 247.00972735798373, 244.17938379094736, 201.81719599208182, 247.00972735798373]
['0980700', 0.0, -17.759286034912716, 247.28023176007594, 244.1

['0980700', 0.007179340659340661, -4.685238778054862, 247.5930916446034, 244.17938379094736, 201.81719599208182, 247.5930916446034]
['0980700', 0.007179340659340661, -4.729860099750623, 247.5930916446034, 244.17938379094736, 201.81719599208182, 247.5930916446034]
['0980700', 0.007179340659340661, -4.774481421446383, 247.54857359614888, 244.17938379094736, 201.81719599208182, 247.54857359614888]
['0980700', 0.007179340659340661, -4.819102743142144, 247.54857359614888, 244.17938379094736, 201.81719599208182, 247.54857359614888]
['0980700', 0.007179340659340661, -4.863724064837905, 247.50400085163568, 244.17938379094736, 201.81719599208182, 247.50400085163568]
['0980700', 0.007179340659340661, -4.908345386533665, 247.50400085163568, 244.17938379094736, 201.81719599208182, 247.50400085163568]
['0980700', 0.007179340659340661, -4.952966708229426, 247.63754941172087, 244.17938379094736, 201.81719599208182, 247.63754941172087]
['0980700', 0.007179340659340661, -4.997588029925186, 247.54857359

In [48]:
df

,time,longitude,latitude,G17_Temp,G17_mean,G17_std,target_G16_Temp


https://erikbern.com/2018/02/15/new-benchmarks-for-approximate-nearest-neighbors.html

In [ ]:
# Get baseline results here with logisic regression and random forest
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import SGDClassifier, Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression
#REGRESSION TREE B/C IF TIME IS IN WINDOW OF ANOMALY IT WILL DO ONE THING OTHERWISE ANOTHER DO ANOTHER THING
#Try KNN apx

# Set up your models here

def model_one():
    pass

def model_two():
    pass

def model_three():
    pass

def model_four():
    pass

def model_five():
    pass

heldout = [0.95, 0.90, 0.75, 0.50, 0.01]
rounds = 20

classifiers = [
    ("SGD", SGDClassifier(max_iter=100)),
    ("ASGD", SGDClassifier(average=True)),
    ("Perceptron", Perceptron()),
    ("Passive-Aggressive I", PassiveAggressiveClassifier(loss='hinge',
                                                         C=1.0, tol=1e-4)),
    ("Passive-Aggressive II", PassiveAggressiveClassifier(loss='squared_hinge',
                                                          C=1.0, tol=1e-4)),
    ("SAG", LogisticRegression(solver='sag', tol=1e-1, C=1.e4 / X.shape[0]))
]

xx = 1. - np.array(heldout)

for name, clf in classifiers:
    print("training %s" % name)
    rng = np.random.RandomState(42)
    yy = []
    for i in heldout:
        yy_ = []
        for r in range(rounds):
            X_train, X_test, y_train, y_test = \
                train_test_split(X, y, test_size=i, random_state=rng)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            yy_.append(1 - np.mean(y_pred == np.array(y_test)))
        yy.append(np.mean(yy_))
    plt.plot(xx, yy, label=name)

plt.legend(loc="upper right")
plt.xlabel("Proportion train")
plt.ylabel("Test Error Rate")
plt.show()

In [ ]:
# Part 4 inplement cross validation here
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.get_n_splits(X)
print(kf)

xx = ['K_fold-1','K_fold-2','K_fold-3','K_fold-4','K_fold-5']
rounds = 20
#count = 0
for train_index, test_index in kf.split(X):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = np.array(X)[np.array(train_index)], np.array(X)[np.array(test_index)]
     y_train, y_test = np.array(y)[np.array(train_index)], np.array(y)[np.array(test_index)]
        
     classifiers = [
     ("ASGD", SGDClassifier(average=True)),
     ("SAG", LogisticRegression(solver='sag', tol=1e-1, C=1.e4 / X.shape[0]))
     ]
     #count+=1

     for name, clf in classifiers:
         print("training %s" % name)
         rng = np.random.RandomState(42)
         yy = []
         for i in heldout:
             yy_ = []
             for r in range(rounds):
                 clf.fit(X_train, y_train)
                 y_pred = clf.predict(X_test)
                 yy_.append(1 - np.mean(y_pred == np.array(y_test)))
             yy.append(np.mean(yy_))
         plt.plot(xx, yy, label=name)

     plt.legend(loc="upper right")
     plt.xlabel("Train-Test Index")
     plt.ylabel("Test Error Rate")
     plt.show()